In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import numpy as np
import gc
import os
import optuna
import sqlite3
import ray
import matplotlib.pyplot as plt
import polars as pl
from optuna.integration import LightGBMPruningCallback
from autogluon.tabular import TabularPredictor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from joblib import Parallel, delayed
from more_itertools import chunked
from functools import reduce
from typing import List
from itertools import combinations

In [2]:
gc.collect()
df_full = pd.read_parquet('./data/l_vm_completa_train.parquet', engine='fastparquet')

In [6]:
# Eliminar de df_full las filas donde la columna A_PREDECIR sea 'N'
df_full = df_full[df_full['A_PREDECIR'] != 'N']
df_full = df_full.drop(columns=['A_PREDECIR'])

In [10]:
# QUiero que busques en df_full los product_id, customer_id que solo tienen ceros en TN
def buscar_productos_solo_ceros(df: pd.DataFrame) -> pd.DataFrame:
    # Agrupar por product_id y customer_id, y sumar los valores de TN
    grouped = df.groupby(['PRODUCT_ID', 'CUSTOMER_ID'])['TN'].sum().reset_index()
    
    # Filtrar aquellos que tienen suma de TN igual a 0
    productos_solo_ceros = grouped[grouped['TN'] == 0]
    
    return productos_solo_ceros
productos_solo_ceros = buscar_productos_solo_ceros(df_full)
print(f"Productos con TN solo ceros: {len(productos_solo_ceros)}")

Productos con TN solo ceros: 0


In [9]:
# Eliminar del df_full los product_id, customer_id que solo tienen ceros en TN
def eliminar_productos_solo_ceros(df: pd.DataFrame, productos_solo_ceros: pd.DataFrame) -> pd.DataFrame:
    # Convertir productos_solo_ceros a un set para una búsqueda más rápida
    productos_set = set(zip(productos_solo_ceros['PRODUCT_ID'], productos_solo_ceros['CUSTOMER_ID']))
    
    # Filtrar el DataFrame original
    df_filtrado = df[~df.set_index(['PRODUCT_ID', 'CUSTOMER_ID']).index.isin(productos_set)]
    
    return df_filtrado
df_full = eliminar_productos_solo_ceros(df_full, productos_solo_ceros)

In [8]:
# Ordenar los productos por PRODUCT_ID y CUSTOMER_ID
productos_solo_ceros = productos_solo_ceros.sort_values(by=['PRODUCT_ID', 'CUSTOMER_ID'])
# Mostrar los primeros 10 productos
print(productos_solo_ceros.head(10))

     PRODUCT_ID  CUSTOMER_ID   TN
84        20001        10085  0.0
141       20001        10142  0.0
165       20001        10166  0.0
167       20001        10168  0.0
173       20001        10174  0.0
176       20001        10177  0.0
190       20001        10191  0.0
209       20001        10210  0.0
215       20001        10216  0.0
218       20001        10219  0.0
